# Convert output of munge_sumstats.py to matlab format

In [1]:
import itertools as it
import pandas as pd
import scipy.io as sio

# complementary bases
COMPLEMENT = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
# bases
BASES = COMPLEMENT.keys()
# true iff strand ambiguous
STRAND_AMBIGUOUS = {''.join(x): x[0] == COMPLEMENT[x[1]]
                    for x in it.product(BASES, BASES)
                    if x[0] != x[1]}
# SNPS we want to keep (pairs of alleles)
VALID_SNPS = {x for x in map(lambda y: ''.join(y), it.product(BASES, BASES))
              if x[0] != x[1] and not STRAND_AMBIGUOUS[x]}
# T iff SNP 1 has the same alleles as SNP 2 (allowing for strand or ref allele flip).
MATCH_ALLELES = {x for x in map(lambda y: ''.join(y), it.product(VALID_SNPS, VALID_SNPS))
                 # strand and ref match
                 if ((x[0] == x[2]) and (x[1] == x[3])) or
                 # ref match, strand flip
                 ((x[0] == COMPLEMENT[x[2]]) and (x[1] == COMPLEMENT[x[3]])) or
                 # ref flip, strand match
                 ((x[0] == x[3]) and (x[1] == x[2])) or
                 ((x[0] == COMPLEMENT[x[3]]) and (x[1] == COMPLEMENT[x[2]]))}  # strand and ref flip
# T iff SNP 1 has the same alleles as SNP 2 w/ ref allele flip.
FLIP_ALLELES = {''.join(x):
                ((x[0] == x[3]) and (x[1] == x[2])) or  # strand match
                # strand flip
                ((x[0] == COMPLEMENT[x[3]]) and (x[1] == COMPLEMENT[x[2]]))
                for x in MATCH_ALLELES}

def _filter_alleles(alleles):
    '''Remove bad variants (mismatched alleles, non-SNPs, strand ambiguous).'''
    ii = alleles.apply(lambda y: y in MATCH_ALLELES)
    return ii

class Logger(object):
    '''
    Lightweight logging.
    TODO: replace with logging module
    '''
    def __init__(self, fh = None):
        self.log_fh = open(fh, 'wb') if fh is not None else None

    def log(self, msg):
        '''
        Print to log file and stdout with a single command.
        '''
        if self.log_fh is not None: print >>self.log_fh, msg
        print(msg)

def read_csv(fh, **kwargs):
    return pd.read_csv(fh, delim_whitespace=True, na_values='.', **kwargs)

def read_sumstats(fh, alleles=False, dropna=True):
    '''Parses .sumstats files. See docs/file_formats_sumstats.txt.'''
    dtype_dict = {'SNP': str,   'Z': float, 'N': float, 'A1': str, 'A2': str}
    compression = 'infer'  # get_compression(fh)
    usecols = ['SNP', 'Z', 'N']
    if alleles:
        usecols += ['A1', 'A2']

    try:
        x = read_csv(fh, usecols=usecols, dtype=dtype_dict, compression=compression)
    except (AttributeError, ValueError) as e:
        raise ValueError('Improperly formatted sumstats file: ' + str(e.args))

    if dropna:
        x = x.dropna(how='any')

    return x

def _read_sumstats(log, fh, alleles=False, dropna=False):
    '''Parse summary statistics.'''
    log.log('Reading summary statistics from {S} ...'.format(S=fh))
    sumstats = read_sumstats(fh, alleles=alleles, dropna=dropna)
    log_msg = 'Read summary statistics for {N} SNPs.'
    log.log(log_msg.format(N=len(sumstats)))
    m = len(sumstats)
    sumstats = sumstats.drop_duplicates(subset='SNP')
    if m > len(sumstats):
        log.log(
            'Dropped {M} SNPs with duplicated rs numbers.'.format(M=m - len(sumstats)))

    return sumstats

def _select_and_log(x, ii, log, msg):
    '''Fiter down to rows that are True in ii. Log # of SNPs removed.'''
    new_len = ii.sum()
    if new_len == 0:
        raise ValueError(msg.format(N=0))
    else:
        x = x[ii]
        log.log(msg.format(N=new_len))
    return x

def _align_alleles(z, alleles):
    '''Align Z1 and Z2 to same choice of ref allele (allowing for strand flip).'''
    try:
        z *= (-1) ** alleles.apply(lambda y: FLIP_ALLELES[y])
    except KeyError as e:
        msg = 'Incompatible alleles in .sumstats files: %s. ' % e.args
        msg += 'Did you forget to use --merge-alleles with munge_sumstats.py?'
        raise KeyError(msg)
    return z

def sumstats_to_mat(reffile, sumstats, out):
    log = Logger();
    
    log.log('Reading reference SNPs from {S} ...'.format(S=reffile))
    ref = pd.read_csv(reffile, delim_whitespace=True)
    ref = ref[['SNP','A1','A2']]
    log.log('Read reference data for {N} SNPs.'.format(N=len(ref)))
    
    # Reading summary statistics
    s = _read_sumstats(log, sumstats, alleles=True, dropna=False)
    s = s.rename(columns={'A1':'A1x','A2':'A2x'})
    
    # Merging summary statistics with reference file
    s = pd.merge(ref, s, how='left', on=['SNP'])
    log.log('After merging with {F}, {N} SNPs remain.'.format(F='reference file', N=len(s)))
    
    # Drop n/a and mismatched alleles
    s = s.dropna(how='any')
    log.log('After dropping n/a, {N} SNPs remain.'.format(F='ref', N=len(s)))
    s = _select_and_log(s, _filter_alleles(s.A1 + s.A2 + s.A1x + s.A2x), Logger(), '{N} SNPs with valid alleles.')
    
    # Flip Z score to match alleles
    s.Z = _align_alleles(s.Z, s.A1 + s.A2 + s.A1x + s.A2x)

    # Align again to the reference (adds nan for missing SNPs)
    s = pd.merge(ref[['SNP']], s[['SNP', 'Z', 'N']], how='left', on=['SNP'])

    # Save result to .mat file
    save_dict = {'zvec':s['Z'].values, 'nvec':s['N'].values}
    sio.savemat(out, save_dict, format='5', do_compression=False, oned_as='column', appendmat=False)
    print('{0} zvec and nvec values, {1} of which are not null, written to {f}'.format(len(s), (~s['Z'].isnull()).sum(), f=out))

reffile = r'1m\1m.ref'
#sumstats_to_mat(reffile, r'BMI.sumstats.gz', 'BMI.mat')
#sumstats_to_mat(reffile, r'scz2.sumstats.gz', 'scz.mat')
#sumstats_to_mat(reffile, r'bip.sumstats.gz', 'bip.mat')
#sumstats_to_mat(reffile, r'tg.sumstats.gz', 'tg.mat')

In [14]:
import glob
import os
import ntpath
for file in glob.glob(r'1m\\sumstats\\*'):
    matfile = os.path.splitext(os.path.splitext(ntpath.basename(file))[0])[0] + ".mat"
    sumstats_to_mat(reffile, file, matfile)

Reading reference SNPs from 1m\1m.ref ...
Read reference data for 1190321 SNPs.
Reading summary statistics from 1m\\sumstats\CHARGE_COG_XXXX.sumstats.gz ...
Read summary statistics for 1217311 SNPs.
After merging with reference file, 1190321 SNPs remain.
After dropping n/a, 1061830 SNPs remain.
1061827 SNPs with valid alleles.
1190321 zvec and nvec values, 1061827 of which are not null, written to CHARGE_COG_XXXX.mat
Reading reference SNPs from 1m\1m.ref ...
Read reference data for 1190321 SNPs.
Reading summary statistics from 1m\\sumstats\DIAGRAM_T2D_2016.sumstats.gz ...
Read summary statistics for 1217311 SNPs.
After merging with reference file, 1190321 SNPs remain.
After dropping n/a, 1165750 SNPs remain.
1165750 SNPs with valid alleles.
1190321 zvec and nvec values, 1165750 of which are not null, written to DIAGRAM_T2D_2016.mat
Reading reference SNPs from 1m\1m.ref ...
Read reference data for 1190321 SNPs.
Reading summary statistics from 1m\\sumstats\GIANT_BMI_2015_EUR.sumstats.gz 

In [4]:
import glob
import os
import ntpath
for file in glob.glob(r'1m\sumstats\CHARGE_*.gz'):
    matfile = os.path.splitext(os.path.splitext(ntpath.basename(file))[0])[0] + ".mat"
    sumstats_to_mat(reffile, file, matfile)
    #print(file)


Reading reference SNPs from 1m\1m.ref ...
Read reference data for 1190321 SNPs.
Reading summary statistics from 1m\sumstats\CHARGE_COG_2015.sumstats.gz ...
Read summary statistics for 1217311 SNPs.
After merging with reference file, 1190321 SNPs remain.
After dropping n/a, 1061830 SNPs remain.
1061827 SNPs with valid alleles.
1190321 zvec and nvec values, 1061827 of which are not null, written to CHARGE_COG_2015.mat


In [ ]:
1+1